In [1]:
import time
import random
import pygame
import numpy as np
import pandas as pd
import subprocess
from screeninfo import get_monitors
import math
import csv
from datetime import datetime
import os
import glob

def calc_speed(spd, dfs, fps, disp):
    #To calculate visual angle for any given display setup
    
    speed = np.array(spd) * np.pi / 180 #numpy arrray of speed in visual angle
    tsp = np.tan(speed)
    ss = np.array(mon_size()[disp][0]) #screen size in cms
    sr = np.array(mon_size()[disp][1])
    sw_r = round(sr[0]/ss[0], 2)
    sh_r = round(sr[1]/ss[1], 2)
        
    sx = int(tsp[0] * dfs * sw_r / fps)
    sy = int(tsp[1] * dfs * sh_r / fps)
    asx = np.arctan(sx * fps / (sw_r * dfs)) * 180 / np.pi
    asy = np.arctan(sy * fps / (sh_r * dfs)) * 180 / np.pi
    
    s_ball = [sx, sy]
    av_ball = [asx, asy]
    
    return s_ball, av_ball

def mon_size():
    #Gets details of all displays connected to system
    
    dml = []
    for m in get_monitors():
        wl = m.width_mm; hl = m.height_mm;
        wp = m.width; hp = m.height;
        dml.append([[round(wl,1), round(hl,1)], [wp, hp]])
    return dml

def calibration_points(rb, disp):
    # Generates fifteen points that are used as start/fixation points for the given display
    
    disp_size = mon_size()[disp]
    ssize = np.array(mon_size()[disp][0]) 
    srez = np.array(mon_size()[disp][1])
    diam = rb*2

    binelh, binerh = [diam, srez[0]-diam]
    bineuv, binedv = [diam, srez[1]-diam]

    midh = math.ceil(srez[0]/2)
    midv = math.ceil(srez[1]/2)

    midlh = math.ceil((midh-diam)/2 + diam)
    midrh = math.ceil((binerh - midh)/2 + midh)
    # midlv = math.ceil((midh-rb)/2 + rb)
    # midrv = math.ceil((bin_edge_h - midh)/2)

    calib_pts = []

    calib_pts.append([binelh, bineuv])
    calib_pts.append([midlh, bineuv])
    calib_pts.append([midh, bineuv])
    calib_pts.append([midrh, bineuv])
    calib_pts.append([binerh, bineuv])

    calib_pts.append([binelh, midv])
    calib_pts.append([midlh, midv])
    calib_pts.append([midh, midv])
    calib_pts.append([midrh, midv])
    calib_pts.append([binerh, midv])

    calib_pts.append([binelh, binedv])
    calib_pts.append([midlh, binedv])
    calib_pts.append([midh, binedv])
    calib_pts.append([midrh, binedv])
    calib_pts.append([binerh, binedv])
    
    dens = ssize/srez
    bin_size = [(binerh-binelh)/4, (binedv - bineuv)/2] #Calculates the distance between each fixation point 
    
    return calib_pts, bin_size

def exit_disp(all_points, correct_clicks, wrong_clicks, tooerc):
    #Displays the results for a second
    #Saves the trajectory of the ball in the successful trials to a csv file in a folder named Trajectories (If that folder doesn't exist, it creates the folder)
    
    #Printing results
    screen.fill((0,0,0))
    font = pygame.font.SysFont(None, 48)
    text = font.render(f"Correct guess = {correct_clicks}", True, (255, 255, 255))
    text2 = font.render(f"Wrong guess = {wrong_clicks}", True, (255, 255, 255))
    text3 = font.render(f" Too hasty = {tooerc}", True, (255, 255, 255))
    screen.blit(text, (width/2 - text.get_width()/2, height/2 - 3 * text.get_height()/2))
    screen.blit(text2, (width/2 - text2.get_width()/2, height/2))
    screen.blit(text3, (width/2 - text3.get_width()/2, height/2 + 3 * text3.get_height()/2))
    pygame.display.flip()

    pygame.time.wait(5000)
    
    #Saving Trajectories
    df = all_points
    traj = pd.DataFrame(df, columns= ['Calibration_point', 'Trajectory_of_ball'])
    now = datetime.now();today = now.strftime("%d-%m-%Y-%H-%M-%S") #Collects current date and time
    
    if not os.path.exists('Trajectories'):
        os.makedirs('Trajectories')
    
    fl_name = 'Trajectories/Trajectory_' + today + '.csv'
    
    # open the file in the write mode
    f = open(fl_name, 'w')

    # write a row to the csv file
    traj.to_csv(fl_name, index=False)
    
    pygame.display.quit()
    pygame.quit()
    exit()

    
# initialize pygame, sound player
pygame.init()
pygame.mixer.init()

deets = pygame.display.Info() #Collects the resolution of your screen 


#----------------EDIT HERE ONLY---------------------------------------------------------------------------------------------------------------------------------

vol = 0.1          #set volume
ntrial = 1         #number of trials
rb = 40            #radius of the ball
app_dur = [3, 7]   #time for which ball is seen
fl_dur = 0.15      #Flicker duration
click_window = 1   #Time after flicker within which the subject must respond 
set_fps = 59       #Set fps of game
rend_stats = True  #render fps and time elapsed 
vas = [15, 15]     #set speed in visual angle/sec 
dfs = 800          #Determine approximte distance from monitor
disp = 0           #Set output display
audio = True       #Whether you want to play audio or not


# define colors 
col_bb = (255, 0, 0)                   #colour of ball when bouncing
col_flk = (180, 0, 0)                 #colour of ball during flicker
col_bg = (0, 0, 0)                     #colour of background
fixation_color = (255, 255, 255)       #colour of ball at the start of trial - fixation point

#----------------------------------------------------------------------------------------------------------------------------------------------------------------


trial_order_l, size_bin = calibration_points(rb, disp)
trial_order = np.array(trial_order_l)
np.random.shuffle(trial_order)

# Calculates the pixel speed based on your choice of visual angle speed. 
# NOTE - you need to specify an approximate distance from screen. 
spd, avas = calc_speed(vas, dfs, set_fps, disp) 

print(size_bin)

# defining width and height of screen - Collected from the info taken of the screen.
infoObject = pygame.display.Info()
width = infoObject.current_w
height = infoObject.current_h
screen_res = (width, height)

if audio:
    right_ans = pygame.mixer.Sound("/home/yramakrishna/DeepLabCut/conda-environments/Codes/Sounds/reward.wav")
    wrong_ans = pygame.mixer.Sound("/home/yramakrishna/DeepLabCut/conda-environments/Codes/Sounds/failure.wav")
    right_ans.set_volume(vol)
    wrong_ans.set_volume(vol)


clock = pygame.time.Clock()

pygame.display.set_caption("GFG Bouncing game")
screen = pygame.display.set_mode((screen_res), pygame.FULLSCREEN, display=disp) #renders it to the entire screen


correct_clicks = 0
wrong_clicks = 0
tooerc = 0
trial = 1

llb, lub = app_dur

timec = pygame.time.get_ticks()
quits = False

last_seen = []
all_points = []

while len(trial_order)>0:
    # Trial runs
    
    speed = list(map(abs, spd))
    
    rand_dir = random.randint(1, 4)
    if rand_dir == 1:
        speed = [spd[0], spd[1]]
    if rand_dir == 2:
        speed = [-spd[0], spd[1]]
    if rand_dir == 3:
        speed = [spd[0], -spd[1]]
    if rand_dir == 4:
        speed = [-spd[0], -spd[1]]
    
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                exit_disp(all_points, correct_clicks, wrong_clicks, tooerc)
    
    pygame.event.clear()
    rspt = random.randint(0, len(trial_order)-1)
    # Define the fixation point coordinates
    fixation = trial_order[rspt]
    print(fixation)
    fixation_x, fixation_y = fixation #randomize this
    fixation_radius = rb
    
    collect_pts = []

    # Draw the fixation point
    pygame.draw.circle(screen, fixation_color, (fixation_x, fixation_y), fixation_radius)

    # Update the display
    pygame.display.flip()

    # Wait for a mouse click on the fixation point to start the trial
    trial_started = False
    while not trial_started:
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    trial_started = True
                if event.key == pygame.K_ESCAPE:
                    exit_disp(all_points, correct_clicks, wrong_clicks, tooerc)
                    
    # define ball
    ball_obj = pygame.draw.circle(surface=screen, color=col_bb, center=fixation, radius=rb)
    
    start_pt = fixation
    
    # speed = [X direction speed, Y direction speed] in pixels per frame
    speed = spd

    limit = random.randint(llb, lub) 
    
    start_time = time.time()
    flick_em1 = random.randint(0,1)
    flick_em = bool(flick_em1)
    flickr = limit-(fl_dur+click_window)
    bound = [start_time + flickr, start_time+flickr+fl_dur]
    
    stptx = min(start_pt[0], width-100)
    stpty = min(start_pt[1], height-100)
    
    random.seed(time.time())
    # If random invisible walls are introduced.
    iwr = width 
    iwl = 0 
    iwd = height 
    iwu = 0 
    click = False #registering clicks
    tooer = False #registering early clicks
    
    # game loop
    while True:
        # event loop

        if (time.time() - start_time) >= limit:
            last_seen = ball_obj.center
            break

        # fill background colour on screen color on screen
        screen.fill(col_bg)

        # move the ball
        # Let center of the ball is (100,100) and the speed is (1,1)
        ball_obj1 = ball_obj.move(speed)
        # Now center of the ball is (101,101)
        # In this way our wall will move

        # if ball goes out of screen then change direction of movement --------------------------------------------------------------------------------------------------------
        if ball_obj.left <= 0 or ball_obj.right >= width:
            speed[0] = -speed[0]
        if ball_obj.top <= 0 or ball_obj.bottom >= height:
            speed[1] = -speed[1]
        
        # # # #ball changes movement randomly -------------------------------------------------------------------------------------------------------------------------------------
        # if ball_obj1.left <= max(iwl, 0): 
        #     speed[0] = -speed[0]
        #     iwr = random.randint(min(ball_obj.right + 300, width), width)
        # if ball_obj1.right >= min(iwr, width):
        #     speed[0] = -speed[0]
        #     iwl = random.randint(0, max(ball_obj.left - 300, 0))
        # if ball_obj1.top <= max(iwu, 0):
        #     speed[1] = -speed[1]
        #     iwd = random.randint(min(ball_obj.bottom + 300, height), height)
        # if ball_obj1.bottom >= min(height,iwd):
        #     speed[1] = -speed[1]
        #     iwu = random.randint(0, max(ball_obj.top - 300, 0))

        ball_obj = ball_obj.move(speed)
        collect_pts.append(ball_obj.center)

        if (time.time() >= bound[0] and time.time() <= bound[1]) and flick_em:
            pygame.draw.circle(surface=screen, color=col_flk, center=ball_obj.center, radius=rb)  
        # draw ball at new centers that are obtained after moving ball_obj
        else:
            pygame.draw.circle(surface=screen, color=col_bb, center=ball_obj.center, radius=rb)
        
        
        timec = pygame.time.get_ticks()/1000
        clock.tick(set_fps)
        fps_disp = clock.get_fps()

        if rend_stats == True:
            font = pygame.font.SysFont('Arial', 20)
            text4 = font.render(f'Time taken: {timec}', True, (255, 255, 0))
            text5 = font.render(f'FPS: {fps_disp}', True, (255, 255, 0))
            screen.blit(text4, (0, 0))
            screen.blit(text5, (250, 0))

        # update screen
        pygame.display.flip()
        
        if time.time() <= (bound[0]):
            for event in pygame.event.get():
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_SPACE:
                            tooer = True
        else:
            if click != True and (time.time() - start_time) <= limit:
                for event in pygame.event.get():
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_SPACE:
                            click = True
        if tooer == True:
            break
        
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    exit_disp(all_points, correct_clicks, wrong_clicks, tooerc)
                    
     
    if tooer == True:
        tooerc += 1
        if audio:
            pygame.mixer.Sound.play(wrong_ans)
    else:
        trial = trial+1
        if click == flick_em:
            correct_clicks += 1
            all_points.append([fixation,collect_pts])
            # print(rspt)
            trial_order = np.delete(trial_order, rspt, 0)
            # print(trial_order)
            if audio:
                pygame.mixer.Sound.play(right_ans)
        else:
            wrong_clicks += 1
            if audio:
                pygame.mixer.Sound.play(wrong_ans)
    

    np.random.shuffle(trial_order)            

    pygame.draw.circle(surface=screen, color=col_bg, center=ball_obj.center, radius=rb)
    pygame.display.flip()
    pygame.time.wait(1000)


    print(flick_em, click)
    print("")
                    
exit_disp(all_points, correct_clicks, wrong_clicks, tooerc)

# pygame.display.quit()
# exit()


pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
[440.0, 460.0]
[520 540]
False False

[520  80]
True True

[  80 1000]
False False

[1400 1000]
False False

[960  80]
False False

[ 80 540]


error: video system not initialized

In [2]:
#To retrieve info from all files in the folder 'Trajectories'

import pandas as pd
import glob
import os

path = os.getcwd() + '/Trajectories'
csv_files = glob.glob(os.path.join(path, "*.csv"))
everything = pd.DataFrame()
appd = []
orig = []

for f in csv_files:
    df = pd.read_csv(f)
    appd.append(df)
    orig.append(f)

#Concatenates everything into a single dataframe
everything = pd.concat(appd, keys=orig, names=['File name', 'Content'])
everything

Calibration_point  \
File name                                          Content                     
/home/yramakrishna/DeepLabCut/conda-environment... 0             [ 960 1000]   
                                                   1             [1400   80]   
                                                   2             [  80 1000]   
/home/yramakrishna/DeepLabCut/conda-environment... 0             [1840   80]   
                                                   1             [1400 1000]   
                                                   2                 [80 80]   

                                                                                           Trajectory_of_ball  
File name                                          Content                                                     
/home/yramakrishna/DeepLabCut/conda-environment... 0        [(973, 1013), (986, 1026), (999, 1039), (1012,...  
                                                   1        [(1387, 67), (1374, 54), (1361, 41), (1348, 28...  
                                                   2        [(93, 987), (106, 974), (119, 961), (132, 948)...  
/home/yramakrishna/DeepLabCut/conda-environment... 0        [(1853, 93), (1866, 106), (1879, 119), (1892, ...  
                                                   1        [(1387, 1013), (1374, 1026), (1361, 1039), (13...  
                                                   2        [(67, 93), (54, 106), (41, 119), (28, 132), (4...